In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/jfk.csv')

In [4]:
dataset.shape

(8757, 1049)

In [5]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,6.0
1,2018-01-01,4,0,3,0,0,1,0,0,1,...,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0
2,2018-01-01,5,0,4,0,0,1,2,3,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [6]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

777

In [7]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 272)

In [8]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

258

In [9]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [10]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 272)
(2190, 272)


In [11]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 258])
torch.Size([2190, 13])
torch.Size([2190, 258])


In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [13]:
bptt = 6

In [14]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [15]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([6, 13]), torch.Size([6, 258]), torch.Size([6, 258]))

In [16]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [17]:
class LSTM(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10):
        super().__init__()
        
        # aggregation
        self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
        self.attachment_matrix.requires_grad = True
        
        
        self.hidden_layer_size = hidden_layer_size
        
        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                    torch.zeros(layers,1,self.hidden_layer_size))
        
        lstm_input = communities + feat_size
        self.lstm = nn.LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers)

        #disaggregation
#         self.linear_1 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.linear_2 = nn.Linear(hidden_layer_size, network_size)


    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)

        
        lstm_out, self.hidden_cell = self.lstm(x.view(len(input_seq) ,1, -1), self.hidden_cell)
        
        predictions = self.linear_2(lstm_out.view(len(input_seq), -1))
#         predictions = F.relu(predictions)
#         predictions = self.linear_2(predictions)
        
        return predictions

In [18]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
            model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                            torch.zeros(layers, 1, model.hidden_layer_size))
            prediction.append(model(seq,feat)[-1])

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [19]:
layers = 1
communities = 24
network_size = len(targetColumns)
feat_size = len(features_cols)

model = LSTM(feat_size = feat_size, hidden_layer_size=communities,
             network_size=network_size, layers=layers,
            communities=communities).to(device)

loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [20]:
epochs = 100


for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
    if i%10 == 1:
        r2, rmse, mae = evaluate(model)
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 1.50980508 r2: 0.386 rmse: 5.230 mae: 1.233
epoch:  11 loss: 1.43684399 r2: 0.472 rmse: 4.504 mae: 1.157
epoch:  21 loss: 1.41309476 r2: 0.452 rmse: 4.670 mae: 1.173
epoch:  31 loss: 1.38317263 r2: 0.498 rmse: 4.282 mae: 1.126
epoch:  41 loss: 1.36921668 r2: 0.512 rmse: 4.156 mae: 1.110
epoch:  51 loss: 1.40990531 r2: 0.515 rmse: 4.131 mae: 1.104
epoch:  61 loss: 1.38272142 r2: 0.506 rmse: 4.206 mae: 1.111
epoch:  71 loss: 1.38073325 r2: 0.506 rmse: 4.214 mae: 1.112
epoch:  81 loss: 1.38124359 r2: 0.507 rmse: 4.200 mae: 1.109
epoch:  91 loss: 1.36289167 r2: 0.511 rmse: 4.166 mae: 1.101
epoch:  99 loss: 1.3631526232


In [21]:
evaluate(model)

(0.5079145930626161, 4.194298, 1.1038326)

In [22]:
attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy()
community_assignment = dict(zip(targetColumns, attachment))
community_assignment

{'1': 1,
 '10': 1,
 '100': 20,
 '101': 19,
 '102': 19,
 '106': 20,
 '107': 22,
 '108': 13,
 '109': 4,
 '11': 15,
 '110': 11,
 '111': 1,
 '112': 23,
 '113': 2,
 '114': 11,
 '115': 12,
 '116': 18,
 '117': 3,
 '118': 6,
 '119': 5,
 '12': 12,
 '120': 21,
 '121': 11,
 '122': 13,
 '123': 22,
 '124': 4,
 '125': 23,
 '126': 20,
 '127': 18,
 '128': 0,
 '129': 0,
 '13': 19,
 '130': 23,
 '131': 22,
 '133': 7,
 '134': 17,
 '135': 11,
 '136': 0,
 '137': 21,
 '138': 15,
 '139': 5,
 '14': 5,
 '140': 19,
 '141': 5,
 '142': 19,
 '143': 6,
 '144': 23,
 '145': 6,
 '146': 4,
 '147': 8,
 '148': 3,
 '149': 12,
 '15': 11,
 '150': 9,
 '151': 7,
 '152': 15,
 '153': 23,
 '154': 20,
 '155': 14,
 '156': 13,
 '157': 6,
 '158': 22,
 '159': 14,
 '16': 20,
 '160': 1,
 '161': 23,
 '162': 23,
 '163': 16,
 '164': 19,
 '165': 18,
 '166': 5,
 '167': 15,
 '168': 8,
 '169': 12,
 '17': 2,
 '170': 5,
 '171': 23,
 '172': 11,
 '173': 20,
 '174': 7,
 '175': 6,
 '176': 21,
 '177': 10,
 '178': 3,
 '179': 22,
 '18': 21,
 '180': 18,

In [23]:
# 20 comm
# 0.505

In [24]:
# 50 comm
# 